In [1]:
import os
import json
import time
import pickle
import requests
import math


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.DataFrame()
df = pd.read_csv('may_june_july.csv', delimiter="|")


In [3]:
# Combine all text 
df['tags'] = df['descr'] + " " + df["title"] + " " + df["cat"]+ " " + df["primary_kw"]+ " " + df["tags"] + " " + df["cat"] 
# Drop unneeded columns
df.drop('id', axis=1, inplace=True)
df.drop('pull_cc', axis=1, inplace=True)
df.drop('cc', axis=1, inplace=True)
df.drop('metav', axis=1, inplace=True)
df.drop('descr', axis=1, inplace=True)
df.drop('title', axis=1, inplace=True)
df.drop('primary_kw', axis=1, inplace=True)
df.drop('cat', axis=1, inplace=True)

In [4]:
# NORMALIZE TO LOG DISTRIBUTION
# Compute Log (freq*impressions/1000)
# Add log column
df['Log'] = df['freq']*df['impressions']/1000

for i, row in df.iterrows():
    cv = math.log(df.iloc[i,3],2)
    df.set_value(i,'Log',cv)

# Drop unneeded column
df.drop('freq', axis=1, inplace=True)
df.drop('impressions', axis=1, inplace=True)

In [5]:
data_mean = df["Log"].mean()
data_mean

9.9790022939737302

In [6]:
data_std = df["Log"].std()
data_std

3.2162998239509824

In [ ]:
#plt.hist(df["Log"])
#plt.show()

In [7]:
# Virality defined as -1 sigma from mean
df['viral'] = np.where(df['Log']<data_mean-data_std, 'notviral', 'viral')
df['viral_num'] = df.viral.map({'notviral':0, 'viral':1})
df.drop('Log', axis=1, inplace=True)
df.head()

,tags,viral,viral_num
0,Giant man with tiny dog alert! The Mountain Fr...,notviral,0
1,FYI: Ice cream sandwiches &gt; all other sandw...,notviral,0
2,"""My mama always said you can tell a lot about ...",notviral,0
3,Let's see if you're a true cheese whiz. Can Yo...,notviral,0
4,The EPA just released first-time guidelines on...,notviral,0


In [8]:
df['tags'].fillna('a', inplace=True)

In [9]:
df.tail()

,tags,viral,viral_num
14292,A former Stanford swimmer who sexually assault...,viral,1
14293,A former Stanford swimmer who sexually assault...,viral,1
14294,A former Stanford swimmer who sexually assault...,viral,1
14295,A definitive ranking of our dirtiest words. Th...,viral,1
14296,"Don't worry, you won't need to know Chandler B...",viral,1


In [10]:
df.shape

(14297, 3)

In [11]:
df.viral.value_counts()

viral       11862
notviral     2435
Name: viral, dtype: int64

In [12]:
X = df.tags
y = df.viral_num
print(X.shape)
print(y.shape)

(14297,)
(14297,)


In [13]:
X.head()

0    Giant man with tiny dog alert! The Mountain Fr...
1    FYI: Ice cream sandwiches &gt; all other sandw...
2    "My mama always said you can tell a lot about ...
3    Let's see if you're a true cheese whiz. Can Yo...
4    The EPA just released first-time guidelines on...
Name: tags, dtype: object

In [14]:
y.head()

0    0
1    0
2    0
3    0
4    0
Name: viral_num, dtype: int64

In [15]:
# split X and y into training and testing sets
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(10722,)
(3575,)
(10722,)
(3575,)


In [16]:
# instantiate the vectorizer
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()

In [17]:
vect

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [18]:
# learn training data vocabulary, then use it to create a document-term matrix
# FOLLOWING CAN BE DONE IN SINGLE STEP:  X_train_dtm = vect.fit_transform(X_train)
vect.fit(X_train)
X_train_dtm = vect.transform(X_train)

In [19]:
X_train_dtm

<10722x15480 sparse matrix of type '<class 'numpy.int64'>'
	with 266669 stored elements in Compressed Sparse Row format>

In [20]:
# transform testing data (using fitted vocabulary) into a document-term matrix
X_test_dtm = vect.transform(X_test)
X_test_dtm

<3575x15480 sparse matrix of type '<class 'numpy.int64'>'
	with 87915 stored elements in Compressed Sparse Row format>

In [21]:
# import and instantiate a Multinomial Naive Bayes model
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()

In [22]:
# train the model using X_train_dtm (timing it with an IPython "magic command")
%time nb.fit(X_train_dtm, y_train)

CPU times: user 6.22 ms, sys: 2.75 ms, total: 8.98 ms
Wall time: 8.08 ms


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [23]:
# make class predictions for X_test_dtm
y_pred_class = nb.predict(X_test_dtm)

In [24]:
# calculate accuracy of class predictions
from sklearn import metrics
metrics.accuracy_score(y_test, y_pred_class)

0.81034965034965034

In [25]:
# print the confusion matrix
metrics.confusion_matrix(y_test, y_pred_class)

array([[ 202,  421],
       [ 257, 2695]])

In [26]:
# print message text for the false positives (non-viral incorrectly classified as viral)
X_test[y_test < y_pred_class]

2356     So much fierce, so little time. Here's What A ...
4046     Robert Czegely has been accused of "gross misc...
3116     Put your JT fandom to the test. Are These Nsyn...
1790     Do you read bad drawings? Can You Identify The...
4015     **prays for ninth movie** Daniel Radcliffe Has...
1094     Hold the door. 53 Thoughts I Had Watching Seas...
3807     Three of the former Libyan dictator s top offi...
153      Seven police officers and two civilians were h...
2410     Tiny home, big clean. 28 Clever Ways To Deep C...
2813     Today's characters: the Malfoys. Here Are The ...
4781     <b>These big rocks in the desert will blow you...
3312     For when you want a little bit of ~everything....
10363    Yes, I want my house to smell like fairy bread...
3895     "Next week will see the debut of Hodor's cousi...
4290     Some Bernie-or-Busters protested Clinton's spe...
1401     Underrated is an UNDERstatement. It's Time To ...
11347    Put all those geography classes to the test. C.

In [27]:
# print message text for the false negatives (Viral incorrectly classified as non-viral)
X_test[y_test > y_pred_class]

11141    Mmmmmmmmmmmmmmmm. 16 Extremely Satisfying Pict...
12571    It wasn't. 17 Things That Will Make Every Indi...
5558     "You're worthy of the last French fry in the b...
10655    Worse still, literally no one is surprised. De...
14152    In 2016, it takes staggering ignorance to thin...
13227    "I am no longer okay with just being a sidekic...
11674    The school has thousands of students, no full-...
11436    "I have always said, I will work after marriag...
9034     TL;DR Poo is done with your shit. Kareena Kapo...
5811     A hot new type of food stack. People On Instag...
1175     Give those old duds new life. 18 No-Sew Ways T...
3153     The pound may be weak, but the memes are stron...
6004     She was a better Trump than Trump. Meryl Stree...
5640     Vote Leave, take back the status quo? Four Bre...
8943     "She just wakes up like that." Aishwarya Rai B...
11646    Pappis for puppies. 9 Pictures Of Doggies At I...
14051    Kya re mamu! Sab changa? Yeh quiz lega kya bhe.

In [28]:
# example false negative
#X_test[3]

In [29]:
# calculate predicted probabilities for X_test_dtm (poorly calibrated)
y_pred_prob = nb.predict_proba(X_test_dtm)[:, 1]
y_pred_prob

array([ 0.65041721,  0.99980255,  0.9999984 , ...,  0.99999913,
        0.99999986,  0.9973585 ])

In [30]:
# calculate AUC
metrics.roc_auc_score(y_test, y_pred_prob)

0.7378497370447219

In [31]:
# import and instantiate a logistic regression model
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()

In [ ]:
# train the model using X_train_dtm
%time logreg.fit(X_train_dtm, y_train)

In [ ]:
# make class predictions for X_test_dtm
y_pred_class = logreg.predict(X_test_dtm)

In [ ]:
# calculate predicted probabilities for X_test_dtm (well calibrated)
y_pred_prob = logreg.predict_proba(X_test_dtm)[:, 1]
y_pred_prob

In [ ]:
# calculate accuracy
metrics.accuracy_score(y_test, y_pred_class)

In [ ]:
# calculate AUC
metrics.roc_auc_score(y_test, y_pred_prob)

In [ ]:
# store the vocabulary of X_train
X_train_tokens = vect.get_feature_names()
len(X_train_tokens)

In [ ]:
# examine the first 50 tokens
print(X_train_tokens[0:50])

In [ ]:
# examine the last 50 tokens
print(X_train_tokens[-50:])

In [ ]:
# Naive Bayes counts the number of times each token appears in each class
nb.feature_count_

In [ ]:
# rows represent classes, columns represent tokens
nb.feature_count_.shape

In [ ]:
# number of times each token appears across all Non-viral Buzzes
non_viral_token_count = nb.feature_count_[0, :]
non_viral_token_count

In [ ]:
# number of times each token appears across all Viral Buzzes
viral_token_count = nb.feature_count_[1, :]
viral_token_count

In [ ]:
# create a DataFrame of tokens with their separate non-viral and viral counts
tokens = pd.DataFrame({'token':X_train_tokens, 'non_viral':non_viral_token_count, 'viral':viral_token_count}).set_index('token')
tokens.head()

In [ ]:
# examine 5 random DataFrame rows
tokens.sample(20, random_state=6)

In [ ]:
# Naive Bayes counts the number of observations in each class
nb.class_count_

In [ ]:
# add 1 to non-viral and viral counts to avoid dividing by 0
tokens['non_viral'] = tokens.non_viral + 1
tokens['viral'] = tokens.viral + 1
tokens.sample(5, random_state=6)

In [ ]:
# convert the non-viral and viral counts into frequencies
tokens['non_viral'] = tokens.non_viral / nb.class_count_[0]
tokens['viral'] = tokens.viral / nb.class_count_[1]
tokens.sample(5, random_state=6)

In [ ]:
# calculate the ratio of viral-to-non-viral for each token
tokens['viral_ratio'] = tokens.viral / tokens.non_viral
tokens.sample(5, random_state=6)

In [ ]:
# examine the DataFrame sorted by viral_ratio
# note: use sort() instead of sort_values() for pandas 0.16.2 and earlier
tokens.sort_values('viral_ratio', ascending=False)

In [ ]:
# look up the viral_ratio for a given token
tokens.loc['stanford', 'viral_ratio']